## 🤍정규표현식 REGEXP
> REGEXP는 기본적으로 패턴 매칭에 사용되며 조건을 설정하는 부분인 **WHERE**나 **HAVING**에서 조건 필터링에 사용<br>
> 유연한 패턴 매칭 : 문자열에 대해 더 복잡하고 정교한 조건을 걸 수 있다.<br>
> 정규 표현식은 매우 강력한 도구지만, 너무 복잡한 패턴을 사용하면 성능 저하가 발생할 수 있기 때문에, 필요에 맞는 패턴을 사용하는 것이 중요
```sql
# email 컬럼에 "gmail.com"을 포함하는 이메일 주소만 status 컬럼을 'active'로 업데이트
UPDATE users SET status = 'active'
WHERE email REGEXP 'gmail\\.com';
```
- `\\.`은 **.** 을 문자 그대로 처리하기 위한 이스케이프 시퀀스
    - 정규 표현식에서 **.** 은 임의의 문자를 의미
- `WHERE phone_number REGEXP '[0-9]'` : 숫자를 포함하는가
- `WHERE email REGEXP '^user_'` : 'user_'로 시작하는가, ^는 문자열의 시작을 의미
- `WHERE email REGEXP 'gmail\\.com$'` : 'gmail.com'으로 끝나는가, $는 문자열의 끝을 의미
- \+ (플러스 기호) : **1회 이상 반복**을 의미하는 수량자
    - [0-9]+: 하나 이상의 숫자 (예: "123", "4567" 등)
- **REGEXP BINARY**를 사용하면 대소문자를 구분하지만 REGEXP만 사용하면 대소문자 구분 없이 패턴을 매칭할 수 있다.

## DB 연동

In [6]:
# pip install mysql-connector-python : DB와 Python 연동
# pip install faker : 가짜 정보 생성
import mysql.connector
from faker import Faker
import random # 파이썬 기본 모듈

# (1) MYSQL 연결
db_connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='0070',
    database='study'
)

# (2) SQL 쿼리를 보내는 객체 생성
cursor = db_connection.cursor()

In [5]:
# (3) MYSQL 연결 끊기
db_connection.commit()
cursor.close()
db_connection.close()

True

In [ ]:
# input
# 100명의 users 더미 데이터 생성
faker = Faker()
for _ in range(100):
    username = faker.user_name()
    email = faker.email()

    sql = "INSERT INTO users(username, email) VALUES(%s, %s)"
    values = (username, email)
    cursor.execute(sql, values)

In [10]:
#select
# user_id 불러오기
cursor.execute("SELECT user_id FROM users")

# fetch...() 계열 함수로 결과를 꺼내면 cousor의 내부 버퍼에 저장된 값을 꺼낸 후 버퍼가 비워진다
valid_user_id = [row[0] for row in cursor.fetchall()]
"""
1개만 select해도 튜플형태로 값이 나와서 [(1,), (2,), (3,)]로 나오기 때문에 요소를 꺼내야함
-> 리스트 컴프리헨션 사용 ( 기존 리스트나 반복 가능한 객체로부터 짧고 간결하게 새로운 리스트를 만드는 문법 )
리스트형 : user_info = [(row[1], row[2]) for row in rows]
딕셔너리 : user_dicts = [{'user_id': row[0], 'name': row[1]} for row in rows]
"""

"\n1개만 select해도 튜플형태로 값이 나와서 [(1,), (2,), (3,)]로 나오기 때문에 요소를 꺼내야함\n-> 리스트 컴프리헨션 사용 ( 기존 리스트나 반복 가능한 객체로부터 짧고 간결하게 새로운 리스트를 만드는 문법 )\n리스트형 : user_info = [(row[1], row[2]) for row in rows]\n딕셔너리 : user_dicts = [{'user_id': row[0], 'name': row[1]} for row in rows]\n"

In [11]:
# 100개의 주문 더미 데이터 생성
for _ in range(100):
    user_id = random.choice(valid_user_id)
    product_name = faker.word()
    quantity = random.randint(1, 10)
    try:
        sql = "INSERT INTO orders(user_id, product_name, quantity) VALUES(%s, %s, %s)"
        values = (user_id, product_name, quantity)
        cursor.execute(sql, values)
    except:
        print("오류 발생")
        pass
db_connection.commit()
cursor.close()
db_connection.close()

## JOIN
1. INNER JOIN : 양쪽 테이블 모두에 일치하는 값이 있는 행
```
SELECT * FROM users
INNER JOIN orders ON users.user_id = orders.user_id;
```
2. LEFT JOIN : A의 모든 행은 결과에 나타나며, B에 매칭이 없으면 NULL이 표시 ( RIGHT JOIN은 반대 )
```
SELECT * FROM A
LEFT JOIN B ON A.user_id = B.user_id;
```
3. FULL JOIN : 두 테이블의 모든 행을 포함하며, 매칭되지 않는 경우 NULL<br>
MySQL에서는 FULL JOIN을 기본적으로 지원하지 않으므로 UNION을 사용하여 구현
```
SELECT * FROM users
LEFT JOIN orders ON users.user_id = orders.user_id
UNION
SELECT * FROM users
RIGHT JOIN orders ON users.user_id = orders.user_id;
```